### Acknowledgment
This script was developed with guidance from Ms. Shilpi Shaw, AI Club Research Institute,
and helper code suggestions generated using ChatGPT.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# define the dataset path
TRAIN_PATH = "/content/drive/Shareddrives/1:1_Krish_Nachnani/2024/Glioblastoma Multiforme -genome data/csv/RFE_features_train_new.csv"

In [ ]:
!pip install lifelines torchtuples

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.5/94.5 kB 6.2 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4031 sha256=c44298b00d7e62666623258f503a25cd99b6fb1245f91df133be374a493b9576
  Stored in directory: /root/.cache/pip/wheels/25/cc/e0/ef2969164144c899fedb22b338f6703e2b9cf46eeebf254991
Successfully built autograd-gamma


In [ ]:
import pandas as pd
import numpy as np
# read the data
data = pd.read_csv(TRAIN_PATH)
data.head()

,TSG101,CEP41,TSNAX-DISC1,TSPAN12,TSPAN32,TTC28,TTC3,TTN,TTYH3,UBE2D4,...,ZNF354C,ZNF367,ZNF483,ZNF503,ZNF510,ZNF563,ZNF568,ZNF572,Disease Free (Months),DFS_STATUS_ENCODED
0,0.3436,1.6391,-2.5606,-0.1230,-0.1573,-0.6779,-0.0468,-0.1866,-0.7818,0.1247,...,0.0206,-0.0436,0.8861,-0.0108,-0.0527,-1.5035,-0.9247,0.1938,8.41,1.0
1,0.2182,0.2580,0.4580,1.2892,1.3483,0.5313,-0.1016,0.9733,0.7186,0.4285,...,0.1452,-0.7585,0.8602,0.7059,-0.7763,-0.3219,-0.6997,-1.4266,2.56,1.0
2,-1.1902,-0.0729,-2.5606,-0.1310,0.5123,0.8338,1.1846,0.6050,0.9829,-0.3769,...,0.5829,0.7443,-0.3714,1.1329,0.5104,-0.3447,-0.5706,-0.9225,1.22,0.0
3,-0.2769,1.1440,-2.5606,1.6190,0.6665,0.4959,0.0749,0.9092,0.6783,2.9144,...,0.4075,0.4088,0.2046,0.8918,0.2540,0.5956,1.8727,-0.3134,13.40,1.0
4,-0.9492,0.3891,-0.2818,0.6458,-0.4419,0.6372,0.2749,0.8224,1.6273,0.6347,...,0.7842,0.5378,0.0599,-2.2485,0.4122,0.7788,0.0517,0.7428,3.19,1.0


Creating deep survival model

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtuples import Model
import torchtuples as tt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from lifelines.utils import concordance_index

In [ ]:
import torch
import torch.nn as nn

# Define the DeepSurv model with additional layers and dropout
class DeepSurv(nn.Module):
    def __init__(self, input_dim, hidden_dims, dropout_prob=0.5):
        super(DeepSurv, self).__init__()
        self.layers = nn.ModuleList()
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout_prob)

        # Create hidden layers with dropout
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            self.layers.append(nn.Linear(prev_dim, hidden_dim))
            prev_dim = hidden_dim

        # Output layer
        self.output_layer = nn.Linear(hidden_dims[-1], 1)

    def forward(self, x):
        # Pass through each hidden layer with ReLU activation and dropout
        for layer in self.layers:
            x = self.relu(layer(x))
            x = self.dropout(x)  # Apply dropout after activation
        return self.output_layer(x)

# Function to get the loss (Negative Log-Likelihood)
def nll_loss(preds, durations, events):
    # Cox proportional hazards loss function
    hazards = preds.squeeze()
    log_hazards = torch.log(torch.cumsum(torch.exp(hazards), dim=0))
    uncensored_loss = hazards - log_hazards
    censored_loss = uncensored_loss * events
    return -torch.sum(censored_loss)


Separate x and y

In [ ]:
DEPENDENT_COLUMNS = ['Disease Free (Months)', 'DFS_STATUS_ENCODED']
GENES = [cols for cols in list(data.columns) if cols not in DEPENDENT_COLUMNS]
#prepare the dataset
X = data[GENES]
y = data[DEPENDENT_COLUMNS]

In [ ]:
X.shape

(73, 50)

Cross validation

In [ ]:
# import numpy as np
# from sklearn.model_selection import KFold
# from tqdm.notebook import tqdm

# #change epoch and learning rate and record mean c-index value in hyperparameter sheet

# HIDDEN_DIMENSION = [128, 64, 32]
# NO_EPOCHS = 100
# LEARNING_RATE = 0.0005

# conco_index = []

# # KFold Cross-validation
# kf = KFold(n_splits=5, shuffle=True, random_state=42)

# for train_index, valid_index in kf.split(X):
#     X_train_fold, X_valid_fold = X.iloc[train_index], X.iloc[valid_index]
#     y_train_fold, y_valid_fold = y.iloc[train_index], y.iloc[valid_index]

#     # Convert data to tensors
#     X_train_tensor = torch.tensor(X_train_fold.values, dtype=torch.float32)
#     y_train_duration = torch.tensor(y_train_fold['Disease Free (Months)'].values, dtype=torch.float32)
#     y_train_event = torch.tensor(y_train_fold['DFS_STATUS_ENCODED'].values, dtype=torch.float32)

#     X_test_tensor = torch.tensor(X_valid_fold.values, dtype=torch.float32)
#     y_test_duration = torch.tensor(y_valid_fold['Disease Free (Months)'].values, dtype=torch.float32)
#     y_test_event = torch.tensor(y_valid_fold['DFS_STATUS_ENCODED'].values, dtype=torch.float32)

#     # define the model and the optimizer
#     input_dim = X.shape[1]
#     model = DeepSurv(input_dim, HIDDEN_DIMENSION)
#     optimizer = optim.Adam(model.parameters(), lr = LEARNING_RATE)

#     # train the model
#     for epoch in tqdm(range(NO_EPOCHS)):
#         model.train()
#         optimizer.zero_grad()

#         # Forward pass
#         preds = model(X_train_tensor)

#         # Compute the loss
#         loss = nll_loss(preds, y_train_duration, y_train_event)
#         loss.backward()
#         optimizer.step()

#     # Evaluate the model
#     model.eval()
#     with torch.no_grad():
#         test_preds = model(X_test_tensor)
#         c_index = concordance_index(y_test_duration, -test_preds.numpy(), y_test_event)
#         conco_index.append(c_index)
#         print(f'Concordance Index: {c_index:.4f}')

# # print evaluations
# print("Cross Validation C-Indexes: {}".format(conco_index))
# print("Mean C-Index: {}".format(np.array(conco_index).mean()))

Save the model

In [ ]:
import os
# Save the entire model to a file
SAVE_MODEL_PATH = "/content/drive/Shareddrives/1:1_Krish_Nachnani/2024/Glioblastoma Multiforme -genome data/DeepSurvModel"
MODEL_NAME = "rfe_DeepSurv_try1.pth"
# save the entire model with architecture
#torch.save(model, os.path.join(SAVE_MODEL_PATH, MODEL_NAME))

In [ ]:
import pandas as pd
from sklearn.model_selection import KFold

# Define parameters for experimentation
epochs_list = [50, 60, 70, 80, 90, 100]
learning_rates = [0.00001, 0.00005, 0.0001, 0.0005, 0.001]
#hidden_layers = [256, 128, 64, 32]
hidden_layers = [128, 64, 32]
#hidden_layers = [128, 64]
#hidden_layers = [64, 32]


# List to store results
results = []

# Variable to track the best model
best_model_state = None
best_c_index = -np.inf  # Initialize with negative infinity
best_params = None  # To store parameters of the best model

# Experiment loop
for epochs in epochs_list:
    for lr in learning_rates:
        conco_index = []  # To store c-index for cross-validation

        # KFold Cross-validation
        kf = KFold(n_splits=5, shuffle=True, random_state=42)

        # KFold Cross-validation
        for train_index, valid_index in kf.split(X):
            X_train_fold, X_valid_fold = X.iloc[train_index], X.iloc[valid_index]
            y_train_fold, y_valid_fold = y.iloc[train_index], y.iloc[valid_index]

            # Convert data to tensors
            X_train_tensor = torch.tensor(X_train_fold.values, dtype=torch.float32)
            y_train_duration = torch.tensor(y_train_fold['Disease Free (Months)'].values, dtype=torch.float32)
            y_train_event = torch.tensor(y_train_fold['DFS_STATUS_ENCODED'].values, dtype=torch.float32)

            X_test_tensor = torch.tensor(X_valid_fold.values, dtype=torch.float32)
            y_test_duration = torch.tensor(y_valid_fold['Disease Free (Months)'].values, dtype=torch.float32)
            y_test_event = torch.tensor(y_valid_fold['DFS_STATUS_ENCODED'].values, dtype=torch.float32)

            # Define the model and optimizer
            input_dim = X.shape[1]
            model = DeepSurv(input_dim, hidden_layers)
            optimizer = optim.Adam(model.parameters(), lr=lr)

            # Train the model
            for epoch in range(epochs):
                model.train()
                optimizer.zero_grad()

                # Forward pass
                preds = model(X_train_tensor)

                # Compute the loss
                loss = nll_loss(preds, y_train_duration, y_train_event)
                loss.backward()
                optimizer.step()

            # Evaluate the model
            model.eval()
            with torch.no_grad():
                test_preds = model(X_test_tensor)
                c_index = concordance_index(y_test_duration, -test_preds.numpy(), y_test_event)
                conco_index.append(c_index)

        # Record the mean concordance index for this configuration
        mean_c_index = np.mean(conco_index)
        results.append({
            'Epochs': epochs,
            'Learning Rate': lr,
            'Mean C-Index': mean_c_index
        })
        print(f'Epochs: {epochs}, Learning Rate: {lr}, Mean C-Index: {mean_c_index:.4f}')

        # Update the best model if this configuration is the best so far
        if mean_c_index > best_c_index:
            best_c_index = mean_c_index
            best_model_state = model.state_dict()  # Save the model's state dictionary
            best_params = {'Epochs': epochs, 'Learning Rate': lr}

# Save the best model to a .pth file
torch.save(model, os.path.join(SAVE_MODEL_PATH, MODEL_NAME))
print(f"Best model saved with C-Index: {best_c_index:.4f} and parameters: {best_params}")

# Convert results to DataFrame and save to CSV
results_df = pd.DataFrame(results)
results_df.to_csv('/content/drive/Shareddrives/1:1_Krish_Nachnani/2024/Glioblastoma Multiforme -genome data/DeepSurv/deepsurv_RFE_results_try1.csv', index=False)
print("Experimentation complete. Results saved to 'deepsurv_RFE_results_try1.csv'.")


Epochs: 50, Learning Rate: 1e-05, Mean C-Index: 0.5888
Epochs: 50, Learning Rate: 5e-05, Mean C-Index: 0.4850
Epochs: 50, Learning Rate: 0.0001, Mean C-Index: 0.5388
Epochs: 50, Learning Rate: 0.0005, Mean C-Index: 0.6384
Epochs: 50, Learning Rate: 0.001, Mean C-Index: 0.5576
Epochs: 60, Learning Rate: 1e-05, Mean C-Index: 0.4954
Epochs: 60, Learning Rate: 5e-05, Mean C-Index: 0.4797
Epochs: 60, Learning Rate: 0.0001, Mean C-Index: 0.5130
Epochs: 60, Learning Rate: 0.0005, Mean C-Index: 0.5838
Epochs: 60, Learning Rate: 0.001, Mean C-Index: 0.5954
Epochs: 70, Learning Rate: 1e-05, Mean C-Index: 0.5248
Epochs: 70, Learning Rate: 5e-05, Mean C-Index: 0.6177
Epochs: 70, Learning Rate: 0.0001, Mean C-Index: 0.5471
Epochs: 70, Learning Rate: 0.0005, Mean C-Index: 0.5829
Epochs: 70, Learning Rate: 0.001, Mean C-Index: 0.5715
Epochs: 80, Learning Rate: 1e-05, Mean C-Index: 0.4442
Epochs: 80, Learning Rate: 5e-05, Mean C-Index: 0.5526
Epochs: 80, Learning Rate: 0.0001, Mean C-Index: 0.4883
Epo

In [ ]:
# import pandas as pd
# import numpy as np
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from sklearn.model_selection import KFold
# from lifelines.utils import concordance_index

# # Define parameters for experimentation
# epochs_list = [50, 60, 70, 80, 90, 100]
# learning_rates = [0.00001, 0.00005, 0.0001, 0.0005, 0.001]
# hidden_layers_list = [
#     [128, 64, 32],
#     [64, 32],
#     [64]
# ]
# dropout_rates = [0.2, 0.3, 0.4, 0.5]

# # List to store results
# results = []

# # Variable to track the best model
# best_model = None
# best_model_state = None
# best_c_index = -np.inf  # Initialize with negative infinity
# best_params = None  # To store parameters of the best model

# # Experiment loop
# for hidden_layers in hidden_layers_list:
#     for dropout_rate in dropout_rates:
#         for epochs in epochs_list:
#             for lr in learning_rates:
#                 conco_index = []  # To store c-index for cross-validation

#                 # KFold Cross-validation
#                 kf = KFold(n_splits=5, shuffle=True, random_state=42)

#                 for train_index, valid_index in kf.split(X):
#                     X_train_fold, X_valid_fold = X.iloc[train_index], X.iloc[valid_index]
#                     y_train_fold, y_valid_fold = y.iloc[train_index], y.iloc[valid_index]

#                     # Convert data to tensors
#                     X_train_tensor = torch.tensor(X_train_fold.values, dtype=torch.float32)
#                     y_train_duration = torch.tensor(y_train_fold['Disease Free (Months)'].values, dtype=torch.float32)
#                     y_train_event = torch.tensor(y_train_fold['DFS_STATUS_ENCODED'].values, dtype=torch.float32)

#                     X_test_tensor = torch.tensor(X_valid_fold.values, dtype=torch.float32)
#                     y_test_duration = torch.tensor(y_valid_fold['Disease Free (Months)'].values, dtype=torch.float32)
#                     y_test_event = torch.tensor(y_valid_fold['DFS_STATUS_ENCODED'].values, dtype=torch.float32)

#                     # Define the model and optimizer
#                     input_dim = X.shape[1]
#                     model = DeepSurv(input_dim, hidden_layers, dropout_rate)
#                     optimizer = optim.Adam(model.parameters(), lr=lr)

#                     # Train the model
#                     for epoch in range(epochs):
#                         model.train()
#                         optimizer.zero_grad()

#                         # Forward pass
#                         preds = model(X_train_tensor)

#                         # Compute the loss
#                         loss = nll_loss(preds, y_train_duration, y_train_event)
#                         loss.backward()
#                         optimizer.step()

#                     # Evaluate the model
#                     model.eval()
#                     with torch.no_grad():
#                         test_preds = model(X_test_tensor)
#                         c_index = concordance_index(y_test_duration, -test_preds.numpy(), y_test_event)
#                         conco_index.append(c_index)

#                 # Record the mean concordance index for this configuration
#                 if conco_index:  # Ensure the list is not empty
#                     mean_c_index = np.mean(conco_index)
#                     results.append({
#                         'Hidden Layers': hidden_layers,
#                         'Dropout Rate': dropout_rate,
#                         'Epochs': epochs,
#                         'Learning Rate': lr,
#                         'Mean C-Index': mean_c_index
#                     })
#                     print(f'Hidden Layers: {hidden_layers}, Dropout Rate: {dropout_rate}, Epochs: {epochs}, '
#                           f'Learning Rate: {lr}, Mean C-Index: {mean_c_index:.4f}')

#                     # Update the best model if this configuration is the best so far
#                     if mean_c_index > best_c_index:
#                         best_c_index = mean_c_index
#                         best_model_state = model.state_dict()  # Save the model's state dictionary
#                         best_model = model
#                         best_params = {
#                             'Hidden Layers': hidden_layers,
#                             'Dropout Rate': dropout_rate,
#                             'Epochs': epochs,
#                             'Learning Rate': lr
#                         }

# # Save the best model to a .pth file
# torch.save(best_model, os.path.join(SAVE_MODEL_PATH, MODEL_NAME))
# print(f"Best model saved with C-Index: {best_c_index:.4f} and parameters: {best_params}")

# # Convert results to DataFrame and save to CSV
# results_df = pd.DataFrame(results)
# results_df.to_csv('/content/drive/Shareddrives/1:1_Krish_Nachnani/2024/Glioblastoma Multiforme -genome data/DeepSurv/deepsurv_RFE_results.csv', index=False)
# print("Experimentation complete. Results saved to 'deepsurv_RFE_results.csv'.")
